# save the images as video
# Basler ace camera

In [1]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import time
import matplotlib.pyplot as plt
import sys
from camcommands import camOpen
from multiprocessing import Process
from threading import Thread

connect camera with OpenCV

In [24]:
# list cameras connected with PC
# 0. set number of cameras to use
cameras_2_use = 2

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
devices = tl_factory.EnumerateDevices() #enumerate devices connected

# 1.1 soft ware issue from pylon. this block may be deleted later if the issue is solved
# here, get instance and enumerate devices until two different cameras are recognized
temp_dev_list = []
if len(devices) == 0:
    raise pylon.RuntimeException("No camera connected, please check the camera connection!")
else:
    for i, dev_info in enumerate(devices):
        temp_dev_list.append(dev_info.GetSerialNumber())
    while len(set(temp_dev_list)) != 2: # number of cameras == 2
        tl_factory = pylon.TlFactory.GetInstance()
        devices = tl_factory.EnumerateDevices()
        temp_dev_list = []
        for i, dev_info in enumerate(devices):
            temp_dev_list.append(dev_info.GetSerialNumber())
        print(temp_dev_list)
        if len(set(temp_dev_list)) == 2:
            break

print(temp_dev_list)


"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""

# 2. check the devices information
if len(devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(devices)) # print total number of devices connected
    for i, dev_info in enumerate(devices): # 
        if dev_info.GetDeviceClass() == 'BaslerGigE': # check if the connected cameras are GigE model. Otherwise software trigger won't work
            print("using %s as IP %s SN %s" % (dev_info.GetModelName(), dev_info.GetIpAddress() ,dev_info.GetSerialNumber()))
        else:
            raise EnvironmentError("no GigE device found")

# 3. create instant array and attach all devices on the Array
cams = pylon.InstantCameraArray(min(len(devices), cameras_2_use)) # create instant
for i, cam in enumerate(cams):
    cam.Attach(tl_factory.CreateDevice(devices[i]))

['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21566554', '21585923']
['21566554', '21585923']
total number of devices: 2
using acA1300-30gc as IP 169.254.91.231 SN 21566554
using acA1300-30gc as IP 169.254.4.51 SN 21585923


In [25]:
# setting camera's parameters
fps = 20 # frame per sec
ExposureTime = 2000 # in µs
Height, width = 962, 1286 # in pixel

for cam in cams:
    cam.Open() # open camera to change the parameter
    print("Setting device ", cam.GetDeviceInfo().GetFriendlyName())
    print(cam.MaxNumBuffer.GetValue(), cam.GetQueuedBufferCount(), cam.MaxNumQueuedBuffer.GetValue())
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    cam.AcquisitionFrameRateAbs.SetValue(fps)
    print(cam.Height.GetValue(), cam.Width.GetValue(), cam.ExposureTimeRaw.GetValue(), cam.AcquisitionFrameRateAbs.GetValue())

    cam.Close() # close camera
    

Setting device  acA1300_SN01 (21566554)
10 0 50
962 1286 2000 20.0
Setting device  acA1300_SN03 (21585923)
10 0 50
962 1286 2000 20.0


In [25]:
# capturing video with multiple cameras simultaneously
video_pos = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    #caminstance.StopGrabbing()
    caminstance.StartGrabbingMax(200)
    with get_writer(os.path.join(file_dir, filename)) as writer:
        print("parent process : %s / process id %s" % (os.getppid(), os.getpid()))
        print("recording start with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(50)
        while caminstance.IsGrabbing():
            try :
                print("working")
                res = caminstance.RetrieveResult(10000, pylon.TimeoutHandling_ThrowException)
                print(res)
            except:
                print("something wrong while recording")
            writer.append_data(res.Array)
            res.Release()
        print("recording finish with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.Close()

cam1 = Process(name="cam1", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[0]), cams[0]))
cam2 = Process(name="cam2", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[1]), cams[1]))

cam1.start()

cam1.join()

cam2.start()


cam2.join()

#while True:
#    time.sleep(2)

parent process : 3662 / process id 3834
parent process : 3662 / process id 3835recording start with acA1300_SN01 (21566554) at 2021-11-09 10:37:32.059545

recording start with acA1300_SN03 (21585923) at 2021-11-09 10:37:32.068841
working
working


In [ ]:
### this is origiinal never change!!

# capturing video with multiple cameras at simultaneously
video_pos = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        print("parent process : %s / process id %s" % (os.getppid(), os.getpid()))
        print("recording start with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(50)
        print("trying camera opening")
        caminstance.Open()
        print(caminstance.Height.GetValue())
        caminstance.Close()

        print("cam opening working")


        while caminstance.IsGrabbing():
            try :
                print("working")
                res = caminstance.RetrieveResult(10000, pylon.TimeoutHandling_ThrowException)
                print(res)
            except:
                print("something wrong while recording")
            writer.append_data(res.Array)
            res.Release()
        print("recording finish with %s at %s" % (caminstance.DeviceInfo.GetFriendlyName(), datetime.now()))


cam1 = Process(name="cam1", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[0]), cams[0]))
cam2 = Process(name="cam2", target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[1]), cams[1]))
cam1.start()
cam2.start()
#cam1.join()
#cam2.join()
#print(f'Process cam1 is alive: {cam1.is_alive()}')
#print(f'Process cam2 is alive: {cam2.is_alive()}')


parent process : 1343 / process id 2619
recording start with acA1300_SN03 (21585923) at 2021-11-04 11:28:56.480740parent process : 1343 / process id 2620

recording start with acA1300_SN01 (21566554) at 2021-11-04 11:28:56.487561
trying camera opening
962trying camera opening

cam opening working
recording finish with acA1300_SN03 (21585923) at 2021-11-04 11:28:56.614802
962
cam opening working
recording finish with acA1300_SN01 (21566554) at 2021-11-04 11:28:57.115444


In [11]:
# capturing video with multiple cameras at simultaneously
filename = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(100)
        while caminstance.IsGrabbing():
            res = caminstance.RetrieveResult(100)
            writer.append_data(res.Array)
            res.Release()

for i, cam in enumerate(cams):
    with get_writer("/Users/dkim/Desktop/basler_camera/recording/%s_multi_cams.avi" % (filename[i] ,) ) as writer:
        cam.StopGrabbing()
        cam.StartGrabbingMax(100)
        while cam.IsGrabbing():
            res = cam.RetrieveResult(100)
            print(res)
            writer.append_data(res.Array)
            #print(res.BlockID)
            res.Release()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c5fec60> >


[swscaler @ 0x7f8727379000] Warning: data is not aligned! This can lead to a speed loss


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8600> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd91e40> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon:

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1286, 962) to (1296, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c5fec60> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >


[swscaler @ 0x7f97b5179000] Warning: data is not aligned! This can lead to a speed loss


<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5d5bc930> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5dd366f0> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5c61e360> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon::CGrabResultPtr *' at 0x7fae5ddd8630> >
<pypylon.pylon.GrabResult; proxy of <Swig Object of type 'Pylon:

In [17]:
"""
working with multiple cameras
for the detail commands, see here: https://docs.baslerweb.com/action-commands.html#action-device-key
short paramer description
Action Device Key: 32-bit key to excute a corresponding action on camera
Action Group Key: 32-bit key for a group of devices (cameras) to excute an action
Action Group Mask: 32-bit key for a subgroup of devies

setting up the connection with multiple cameras and checking the parameters
"""

# 1. set action key, group key and group mask. As I'm using one group and all cameras in this group working simaltaneously,
# I did not set multiple action key, group key and group mask
action_key = 0x4711
group_key = 0x112233
group_mask = pylon.AllGroupMask

# 2. Initiate automatic configuration by registering ActionTriggerConfiguration.
for cam in cams:
    cam.RegisterConfiguration(
        pylon.ActionTriggerConfiguration(action_key, group_key, group_mask),
        pylon.RegistrationMode_Append,
        pylon.Cleanup_Delete
        )

# 3. Create a suitable ActionCommand object. For that a GigETransportLayer object is needed.
gige_tl = tl_factory.CreateTl('BaslerGigE')

# 4. Using default value of "255.255.255.255" for fourth parameter 'broadcastAddress'.
act_cmd = gige_tl.ActionCommand(action_key, group_key, group_mask)